<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
%matplotlib inline
import os
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import datasets
import tensorflow as tf
from sklearn.preprocessing import normalize
import faiss
import numpy as np
from functools import partial

from tensorflow.keras import mixed_precision

import numpy as np
import json

from matplotlib import pylab as plt
import seaborn as sns

In [ ]:
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
##Delcare filepaths

#dataset directory
base_data_dir = "../data/subset"

#base dataset
data_fname = f"{base_data_dir}/case_info.json"

#embeddings dataset
embeddings_model_name="allenai/specter"
embedding_type = "specter_pooled"
embedding_dataset_dir = f"{base_data_dir}/{embedding_type}_embeddings_dataset"

#classification dataset
clf_dataset_dir = f"{base_data_dir}/{embedding_type}_clf_dataset"
data_files = {
    "train": f"{base_data_dir}/train_map.csv", 
    "validation": f"{base_data_dir}/val_map.csv", 
    "test": f"{base_data_dir}/test_map.csv",}

#model
BATCH_SIZE= 128
STEP_PER_EPOCH = 1000
VAL_STEPS = 100

models_dir = "../models"
model_name = "specter_pooled_clf_model"
model_checkpoint = f"{models_dir}/{model_name}"
model_log_fname = f"{models_dir}/logs/{model_name}.csv"
model_png = f"{models_dir}/logs/{model_name}.png"

In [ ]:
def load_text(examples):
    """Create text features from head matter and opinion text for a batch of examples"""
    
    batch = [item[0] + "\n" + item[1] for item in zip(examples['head_matter'], examples['opinion_text'])]
    return {"text": batch}

def load_embeddings(examples, tokenizer, model, embedding_type):
    """Tokenize and load embeddings from given huggingface pretrained model"""
    
    tokenized = tokenizer(examples["text"],
                          return_tensors="tf",
                          padding=True,
                          truncation=True,
                          max_length=512
                         )
    if embedding_type == "pooled":
        embeddings = {'embeddings': normalize(model(**tokenized)[1].numpy())}
    else:
        embeddings = {'embeddings': normalize(model(**tokenized)[0][:,0, :].numpy())}
    return embeddings


def load_embeddings_dataset(
    dataset_dir, data_fname, embedding_type, embedding_model="allenai/specter",
    num_proc=15, batch_size=256, faiss_device=0, keep_in_memory=False):
    """Load embeddings dataset and create faiss index on embeddings column"""
    
    if os.path.isfile(f"{dataset_dir}/state.json"):
        print("Found existing embeddings. loading from disk ...")
        dataset = datasets.Dataset.load_from_disk(dataset_dir, keep_in_memory=keep_in_memory)
        if os.path.isfile(f"{dataset_dir}/embeddings.faiss"):
            print("Found existing fiass index. loading from disk ...")
            dataset.load_faiss_index("embeddings", f"{dataset_dir}/embeddings.faiss")
        else:
            print("No fiass index found. creating and saving new index to disk ...")
            dataset.add_faiss_index(column="embeddings", device=faiss_device, metric_type=faiss.METRIC_INNER_PRODUCT)
            dataset.save_faiss_index("embeddings", f"{dataset_dir}/embeddings.faiss")
    else:
        print("No existing embeddings found. Creating and saving to disk ...")
        tokenizer = AutoTokenizer.from_pretrained(embedding_model)
        model = TFAutoModel.from_pretrained(embedding_model, from_pt=True)
        
        print("Loading dataset and text column ...")
        dataset = datasets.load_dataset("json", data_files=data_fname, split=datasets.splits.Split("train"))
        exclude_columns = ["jurisdiction_id","court_id","decision_date", "head_matter","opinion_text","citation_ids"]
        dataset = dataset.map(load_text, batched=True, num_proc=num_proc, remove_columns=exclude_columns)
        print("Loading embeddings ...")
        embedder = partial(load_embeddings, tokenizer=tokenizer, model=model, embedding_type=embedding_type.split("_")[-1])
        dataset = dataset.map(embedder, batched=True, batch_size=batch_size)
        print(f"Saving Dataset to disk at {dataset_dir}")
        dataset.save_to_disk(dataset_dir)
        print("Creating new fiass index and saving to disk ...")
        dataset.add_faiss_index(column="embeddings", device=faiss_device, metric_type=faiss.METRIC_INNER_PRODUCT)
        dataset.save_faiss_index("embeddings", f"{dataset_dir}/embeddings.faiss")
    return dataset

In [ ]:
embeddings_dataset = load_embeddings_dataset(
    embedding_dataset_dir,
    data_fname,
    embedding_type,
    embedding_model=embeddings_model_name)

In [ ]:
def load_clf_embeddings(examples):
    """load embeddings into classification dataset."""
    
    return {
        "case_embedding": embeddings_dataset[examples["idx"]]["embeddings"],
        "citation_embedding": embeddings_dataset[examples["citation_idx"]]["embeddings"],
        }


def generator_from_dataset(dataset):
    """Create a generator from a huggignface dataset."""
    
    def _gen():
        for item in dataset:
            features = (item["case_embedding"], item["citation_embedding"])
            yield features, item['label']
    return _gen


def tf_dataset_from_dataset(dataset):
    """Create a tensorflow dataset from a huggingface dataset using a generator."""
    
    dataset_generator = generator_from_dataset(dataset)
    tfdataset = tf.data.Dataset.from_generator(
        dataset_generator,
        output_signature=(
         (tf.TensorSpec(shape=(768,), dtype=tf.float32),
          tf.TensorSpec(shape=(768,), dtype=tf.float32),
         ),
         tf.TensorSpec(shape=(None), dtype=tf.int32))
    )
    tfdataset = tfdataset.apply(tf.data.experimental.assert_cardinality(len(dataset)))
    return tfdataset

def shuffle_batch_repeat(dataset, batch_size=64, shuffle=False):
    """Batch shuffle and repeat a tensorflow dataset infinitely."""
    
    dataset = dataset.repeat()
    if shuffle:
        dataset = dataset.shuffle(batch_size*4)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


def load_clf_datasets(
    dataset_dir, data_files, batch_size=64,
    embeddings_dataset=None, num_proc=15):
    """Create tensorflow dataset with pairs for embeddings and a label for classification."""
    
    if os.path.isfile(f"{dataset_dir}/dataset_dict.json"):
        print("Found existing dataset dict. loading from disk ...")
        dataset = datasets.DatasetDict.load_from_disk(dataset_dir, keep_in_memory=True)
        dataset.set_format(type='tensorflow', columns=['case_embedding', 'citation_embedding', 'label'])
        
        train_dataset = dataset['train']
        val_dataset = dataset['validation']
        test_dataset = dataset['test']
        
    elif embeddings_dataset is not None:
        print("Found data files and embeddings dataset. Creating new clf dataset and saving to disk ...")
        dataset = datasets.load_dataset("csv", data_files=data_files)
        print("Mapping embeddings to clf dataset ...")        
        dataset = dataset.map(load_clf_embeddings, batched=True, num_proc=num_proc)
        print(f"Saving Dataset to disk at {dataset_dir}")
        dataset.save_to_disk(dataset_dir)
        dataset.set_format(type='tensorflow', columns=['case_embedding', 'citation_embedding', 'label'])
        
        train_dataset = dataset['train']
        val_dataset = dataset['validation']
        test_dataset = dataset['test']

    print("loading tensorflow datasets ...")
    train_dataset = tf_dataset_from_dataset(train_dataset)
    val_dataset = tf_dataset_from_dataset(val_dataset)
    test_dataset = tf_dataset_from_dataset(test_dataset)
    
    print("batching and shuffling tensorflow datasets ... ")
    train_dataset = shuffle_batch_repeat(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = shuffle_batch_repeat(val_dataset, batch_size=batch_size,)
    test_dataset = shuffle_batch_repeat(test_dataset, batch_size=batch_size,)
    
    return train_dataset, val_dataset, test_dataset


In [ ]:
train_dataset, val_dataset, test_dataset = load_clf_datasets(
    clf_dataset_dir, data_files, batch_size=BATCH_SIZE, embeddings_dataset=embeddings_dataset)

In [ ]:
def load_model(model_name, hidden_dim=256, dropout=0.2):
    """Create a classification model for paired embedding inputs"""
    
    case_input = tf.keras.layers.Input(shape=(768,), dtype=tf.float32, name="case_input")
    citation_input = tf.keras.layers.Input(shape=(768,), dtype=tf.float32, name="citation_input")

    case_representation = tf.keras.layers.BatchNormalization()(case_input)
    citation_representation = tf.keras.layers.BatchNormalization()(citation_input)

    case_representation = tf.keras.layers.Dense(hidden_dim, activation="relu")(case_representation)

    citation_representation = tf.keras.layers.Dense(hidden_dim, activation="relu")(citation_representation)

    case_representation = tf.keras.layers.Dropout(dropout,)(case_representation)
    citation_representation = tf.keras.layers.Dropout(dropout,)(citation_representation)

    case_representation = tf.keras.layers.Dense(hidden_dim, activation="relu")(case_representation)
    citation_representation = tf.keras.layers.Dense(hidden_dim, activation="relu")(citation_representation)
    
    sims = tf.keras.layers.Dot(axes=1, normalize=True)([case_representation, citation_representation])
    concatenated = tf.keras.layers.Concatenate()([case_representation, citation_representation])

    def shared_stack(prev_input):
        return tf.keras.models.Sequential(
            [tf.keras.layers.BatchNormalization(),
             tf.keras.layers.Dense(hidden_dim, activation="relu"),
             tf.keras.layers.Dropout(dropout)
            ])(prev_input) 
    concatenated = shared_stack(concatenated)
    concatenated = shared_stack(concatenated)
    concatenated = shared_stack(concatenated)
    output = tf.keras.layers.Dense(1, dtype=tf.float32, activation="sigmoid")(concatenated)
    output = tf.keras.layers.Average()([sims,output])
    clf_model = tf.keras.models.Model(inputs=[case_input, citation_input], outputs=[output], name=model_name)
    loss= tf.keras.losses.BinaryCrossentropy()
    acc = tf.keras.metrics.BinaryAccuracy()
    clf_model.compile(loss=loss, metrics=[acc],  optimizer="adam")
    clf_model.summary()
    return clf_model

In [ ]:
clf_model = load_model(model_name)

In [ ]:
tf.keras.utils.plot_model(
    clf_model,
    model_png,
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=50,
)

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
             tf.keras.callbacks.ModelCheckpoint(model_checkpoint, save_best_only=True,),
             tf.keras.callbacks.CSVLogger(model_log_fname),
             tf.keras.callbacks.ReduceLROnPlateau(patience=2, verbose=1)
            ]
             

In [ ]:
BATCH_SIZE= 128
STEP_PER_EPOCH = 1000
VAL_STEPS = 100
NUM_EPOCHS = 3

In [ ]:
history = clf_model.fit(
    train_dataset,
    steps_per_epoch=STEP_PER_EPOCH,
    validation_data=val_dataset,
    validation_steps=VAL_STEPS,
    verbose=1,
    epochs=NUM_EPOCHS,
    callbacks=callbacks
)
clf_model.save(model_checkpoint)